In [1]:
# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# visualize 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

# working with dates
from datetime import datetime

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt

#important import 
import pandas as pd
import numpy as np
import os
import wrangle as w

In [2]:
#Acquire superstore data from codeup database
df = w.get_superstore_data()

In [3]:
df.head()

,Category ID,Product ID,Customer ID,Order ID,Order Date,Ship Date,Ship Mode,Segment,Country,City,...,Postal Code,Sales,Quantity,Discount,Profit,Region ID,Customer Name,Product Name,Category,Sub-Category
0,10,OFF-AP-10000576,AA-10315,CA-2017-147039,2017-06-29,2017-07-04,Standard Class,Consumer,United States,Minneapolis,...,55407.0,362.940,3.0,0.0,90.7350,3,Alex Avila,Belkin 7 Outlet SurgeMaster II,Office Supplies,Appliances
1,9,OFF-BI-10004654,AA-10315,CA-2017-147039,2017-06-29,2017-07-04,Standard Class,Consumer,United States,Minneapolis,...,55407.0,11.540,2.0,0.0,5.7700,3,Alex Avila,Avery Binding System Hidden Tab Executive Styl...,Office Supplies,Binders
2,5,OFF-ST-10001590,AA-10315,CA-2015-121391,2015-10-04,2015-10-07,First Class,Consumer,United States,San Francisco,...,94109.0,26.960,2.0,0.0,7.0096,2,Alex Avila,Tenex Personal Project File with Scoop Front D...,Office Supplies,Storage
3,5,OFF-ST-10001325,AA-10375,CA-2016-126613,2016-07-10,2016-07-16,Standard Class,Consumer,United States,Mesa,...,85204.0,16.768,2.0,0.2,1.4672,2,Allen Armold,Sterilite Officeware Hinged File Box,Office Supplies,Storage
4,6,FUR-FU-10003347,AA-10375,CA-2015-140921,2015-02-03,2015-02-05,First Class,Consumer,United States,Omaha,...,68104.0,28.400,2.0,0.0,11.0760,3,Allen Armold,Coloredge Poster Frame,Furniture,Furnishings


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1734 entries, 0 to 1733
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Category ID    1734 non-null   int64  
 1   Product ID     1734 non-null   object 
 2   Customer ID    1734 non-null   object 
 3   Order ID       1734 non-null   object 
 4   Order Date     1734 non-null   object 
 5   Ship Date      1734 non-null   object 
 6   Ship Mode      1734 non-null   object 
 7   Segment        1734 non-null   object 
 8   Country        1734 non-null   object 
 9   City           1734 non-null   object 
 10  State          1734 non-null   object 
 11  Postal Code    1734 non-null   float64
 12  Sales          1734 non-null   float64
 13  Quantity       1734 non-null   float64
 14  Discount       1734 non-null   float64
 15  Profit         1734 non-null   float64
 16  Region ID      1734 non-null   int64  
 17  Customer Name  1734 non-null   object 
 18  Product 

In [5]:
#Checking the nulls
df.isnull().sum().sum()

0

In [6]:
#Cleaning superstore data
df = w.prep_superstore(df, 'Order Date', 'Ship Date')

In [7]:
#Check to see if function rename the columns correctly
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1734 entries, 2014-01-04 to 2017-12-30
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_id     1734 non-null   object        
 1   customer_id    1734 non-null   object        
 2   order_id       1734 non-null   object        
 3   ship_date      1734 non-null   datetime64[ns]
 4   ship_mode      1734 non-null   object        
 5   segment        1734 non-null   object        
 6   country        1734 non-null   object        
 7   city           1734 non-null   object        
 8   state          1734 non-null   object        
 9   postal_code    1734 non-null   float64       
 10  sales          1734 non-null   float64       
 11  quantity       1734 non-null   float64       
 12  discount       1734 non-null   float64       
 13  profit         1734 non-null   float64       
 14  customer_name  1734 non-null   object        
 15  pro